In [10]:
import pandas as pd

In [30]:
data = pd.read_json('./putuo.json')
price_gap = 0.9

In [31]:
data_clean = data[['标题', '总价', '每平方售价', '建筑面积', '产权年限', '建造时间', '小区名称', '梯户比例', '所在楼层','链家编号']].rename(index=str, columns={'标题': 'title', '总价': 'total_price', '每平方售价': 'price', '产权年限' : 'period', '建造时间': 'time', '小区名称': 'community', '梯户比例': 'neighbours', '所在楼层': 'floor', '建筑面积':'area','链家编号':'url'})

In [32]:
data_clean.total_price = data_clean.total_price.map(lambda x: x.strip('万')).astype(float)
data_clean.price = data_clean.price.map(lambda x: x.strip('元/平米')).astype(float)
data_clean.url = data_clean.url.map(lambda x: 'https://sh.lianjia.com/ershoufang{}.html'.format(x))


In [36]:
data_clean.head()

,title,total_price,price,area,period,time,community,neighbours,floor,url
0,芝巷小区 1室1厅 275万,275.0,81846.0,33.6㎡,未知,1995年建/板楼,芝巷小区,一梯六户,中楼层 (共6层),https://sh.lianjia.com/ershoufang/107101188634...
1,小区环境好 房东诚意出售 看房方便,212.0,34079.0,62.21㎡,未知,1996年建/板楼,桃浦一村海棠苑,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101004510...
2,本房属商品房。房子南北套，厨卫厅房全明,285.0,48306.0,59㎡,未知,1996年建/板楼,真光第四小区,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101184938...
3,内环内275万双南两房，出入方便还有谁,275.0,54348.0,50.6㎡,未知,1978年建/板楼,普陀四村,一梯五户,高楼层 (共5层),https://sh.lianjia.com/ershoufang/107101166709...
4,此房满五年唯一，税费较少，采光通风比较好,285.0,56559.0,50.39㎡,未知,1998年建/板楼,曹杨花苑,一梯两户,中楼层 (共6层),https://sh.lianjia.com/ershoufang/107101180682...


In [37]:
print('总的房源数目: {}'.format(len(data_clean)))

总的房源数目: 1202


In [38]:
community_price_avg = data_clean.groupby('community', as_index=False).price.agg('mean').rename(columns={'price': 'community_avg_price'})
community_price_avg.head()

,community,community_avg_price
0,万航渡后路87号,74735.0
1,三源路162弄,65390.0
2,上青佳园,37508.0
3,世纪之门,78269.0
4,东新支路55弄,63690.5


In [39]:
print('总的小区数目: {}'.format(len(community_price_avg)))

总的小区数目: 263


In [40]:
data_clean = data_clean.merge(community_price_avg, how='left', on='community')

In [41]:
data_clean.head()

,title,total_price,price,area,period,time,community,neighbours,floor,url,community_avg_price
0,芝巷小区 1室1厅 275万,275.0,81846.0,33.6㎡,未知,1995年建/板楼,芝巷小区,一梯六户,中楼层 (共6层),https://sh.lianjia.com/ershoufang/107101188634...,80655.333333
1,小区环境好 房东诚意出售 看房方便,212.0,34079.0,62.21㎡,未知,1996年建/板楼,桃浦一村海棠苑,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101004510...,38972.500000
2,本房属商品房。房子南北套，厨卫厅房全明,285.0,48306.0,59㎡,未知,1996年建/板楼,真光第四小区,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101184938...,47727.272727
3,内环内275万双南两房，出入方便还有谁,275.0,54348.0,50.6㎡,未知,1978年建/板楼,普陀四村,一梯五户,高楼层 (共5层),https://sh.lianjia.com/ershoufang/107101166709...,62971.250000
4,此房满五年唯一，税费较少，采光通风比较好,285.0,56559.0,50.39㎡,未知,1998年建/板楼,曹杨花苑,一梯两户,中楼层 (共6层),https://sh.lianjia.com/ershoufang/107101180682...,55507.750000


In [45]:
fruit = data_clean[data_clean['price'] < data_clean['community_avg_price'] * 0.9]
print('total: {}, left: {}'.format(len(data_clean), len(fruit)))
fruit

total: 1202, left: 83


,title,total_price,price,area,period,time,community,neighbours,floor,url,community_avg_price
1,小区环境好 房东诚意出售 看房方便,212.0,34079.0,62.21㎡,未知,1996年建/板楼,桃浦一村海棠苑,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101004510...,38972.500000
3,内环内275万双南两房，出入方便还有谁,275.0,54348.0,50.6㎡,未知,1978年建/板楼,普陀四村,一梯五户,高楼层 (共5层),https://sh.lianjia.com/ershoufang/107101166709...,62971.250000
5,东边套全明 通风采光佳 近菜市场 配套成熟,220.0,45380.0,48.48㎡,70年,1954年建/板楼,宜川二村,一梯四户,中楼层 (共5层),https://sh.lianjia.com/ershoufang/107101225187...,50850.500000
15,小区大一房 厅有两扇窗子 户型正气 厨卫朝北徐装修,225.0,42270.0,53.23㎡,未知,1996年建/板楼,真光第四小区,一梯两户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107100766579...,47727.272727
22,紧紧依靠枫桥路地铁站。多爬层楼比多走远走一大截路强,258.0,48297.0,53.42㎡,70年,未知年建/板楼,北枫桥园,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107100144465...,54290.400000
24,3.4.11号线旁 满五年唯一内环内普通住宅，南北通透,236.0,59914.0,39.39㎡,未知,1975年建/板楼,中山北路2590号,一梯四户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101250679...,67142.000000
73,近7号线新村路 精装修两房 房龄新 看房方便 生活便利,299.0,41644.0,71.8㎡,未知,1995年建/板楼,黄陵小区,一梯三户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107100647571...,53223.545455
96,春天花会开 好房等你来 曹杨九村南北通 业主诚售！,240.0,50210.0,47.8㎡,70年,1978年建/板楼,曹杨九村,一梯六户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107100522203...,56886.000000
123,高楼层 采光好 视野开阔 单价低 总价低 一南一北,222.0,44153.0,50.28㎡,未知,1983年建/板楼,章家巷小区,一梯五户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107101047398...,50485.764706
138,业主诚售 全明两房 卧室都是两窗 售后公房 看房方便,288.0,47121.0,61.12㎡,未知,1985年建/板楼,太浜巷小区,一梯六户,高楼层 (共6层),https://sh.lianjia.com/ershoufang/107100145027...,58073.727273
